In [34]:
import pandas as pd
import numpy as np
from numba import jit
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.trading import TradingClient
from alpaca.trading.requests import GetCalendarRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.data.enums import Adjustment

from typing import List, Tuple, Optional

import matplotlib.pyplot as plt
from collections import defaultdict, deque
import numpy as np
import toml
import os
import math

from datetime import datetime
from datetime import datetime, timezone, timedelta

from tqdm import tqdm

# from Trade import Trade, export_trades_to_csv
from TradePosition import TradePosition, SubPosition, export_trades_to_csv
from TouchArea import TouchArea, TouchAreaCollection

from dotenv import load_dotenv

STANDARD_DATETIME_STR = '%Y-%m-%d %H:%M:%S'
ROUNDING_DECIMAL_PLACES = 10  # Choose an appropriate number of decimal places

load_dotenv(override=True)
livepaper = os.getenv('LIVEPAPER')
config = toml.load('../config.toml')

# Replace with your Alpaca API credentials
API_KEY = config[livepaper]['key']
API_SECRET = config[livepaper]['secret']


trading_client = TradingClient(API_KEY, API_SECRET)

In [35]:
def is_security_marginable(symbol: str) -> bool:
    try:
        asset = trading_client.get_asset(symbol)
        return asset.marginable
    except Exception as e:
        print(f"Error checking marginability for {symbol}: {e}")
        return False
    
# # testing purposes
# def is_security_marginable(symbol: str) -> bool:
#     marginable_symbols = {'NVDA', 'MSFT', 'AAPL', 'TJX', 'JNJ', 'AMZN', 'TSLA', 'AZN'}
#     return symbol in marginable_symbols

# for symbol in {'NVDA', 'MSFT', 'AAPL', 'TJX', 'JNJ', 'AMZN', 'TSLA', 'AZN'}:
#     print(symbol, is_security_marginable(symbol))

In [36]:
def get_market_hours(dates):
    """
    Parameters:
    dates (list of str): List of dates in 'YYYY-MM-DD' format.

    Returns:
    
    dict: Dictionary with dates as keys and (market_open, market_close) tuples as values.
    """
    start_date = min(dates)
    end_date = max(dates)
    
    calendar_request = GetCalendarRequest(start=start_date, end=end_date)
    calendar = trading_client.get_calendar(calendar_request)
    
    market_hours = {str(day.date): (day.open, day.close) for day in calendar}
    return market_hours

In [37]:
def calculate_dynamic_levels(df:pd.DataFrame, ema_short=9, ema_long=20):
    """
    Calculate VWAP and EMAs for the dataframe.
    
    :param df: DataFrame with 'close', 'high', 'low', and 'volume' columns
    :param ema_short: Period for the short EMA (default 9)
    :param ema_long: Period for the long EMA (default 20)
    :return: DataFrame with additional columns for VWAP and EMAs
    """
    
    assert 'vwap' in df.columns
    
    # Calculate EMAs
    # df[f'EMA{ema_short}'] = df['close'].ewm(span=ema_short, adjust=False).mean()
    # df[f'EMA{ema_long}'] = df['close'].ewm(span=ema_long, adjust=False).mean()
    
    # Calculate a combined central value
    # df['central_value'] = (df['vwap'] + df[f'EMA{ema_short}'] + df[f'EMA{ema_long}']) / 3
    # df['central_value'] = df['vwap']
    # df['central_value'] = df[f'EMA{ema_short}']
    # df['central_value'] = df[f'EMA{ema_long}']
    
    # 1
    # df['central_value'] = df['close'].ewm(span=26, adjust=True).mean()
    
    # 2
    # df['central_value'] = df['close'].ewm(span=26, adjust=False).mean()
    
    # 3
    # halflife = '26min'  # 26 minutes
    # df['central_value'] = df['close'].ewm(
    #     halflife=halflife,
    #     times=df.index.get_level_values('timestamp'),
    #     adjust=True
    # ).mean()

    # #4
    # span = 26 # span of 26 = 9.006468342000588min
    # alpha = 2 / (span + 1)
    # halflife = np.log(2) / np.log(1 / (1 - alpha))
    # halflife_str = f"{halflife}min"

    # df['central_value'] = df['close'].ewm(
    #     halflife=halflife_str,
    #     times=df.index.get_level_values('timestamp'),
    #     adjust=True
    # ).mean()
    
    #45
    span = 26 # span of 26 = 9.006468342000588min
    alpha = 2 / (span + 1)
    halflife = np.log(2) / np.log(1 / (1 - alpha))
    halflife_str = f"{halflife}min"

    df['central_value'] = df['close'].ewm(
        halflife=halflife_str,
        times=df.index.get_level_values('timestamp'),
        adjust=True
    ).mean()
    df['central_value'] = (df['vwap'] + df['central_value']*2) / 3

In [38]:
@jit(nopython=True)
def process_touches(touches, prices, touch_area_lower, touch_area_upper, level, level_lower_bound, level_upper_bound, is_long, min_touches):
    consecutive_touches = np.full(min_touches, -1, dtype=np.int64)
    count = 0
    prev_price = None
    for i in range(len(prices)):
        price = prices[i]
        is_touch = (prev_price is not None and 
                    ((prev_price < level <= price) or (prev_price > level >= price)) or 
                    (price == level))
        
        if level_lower_bound <= price <= level_upper_bound:
            if is_touch:
                consecutive_touches[count] = touches[i]
                count += 1
                if count == min_touches:
                    return consecutive_touches
        else:
            buy_price = touch_area_upper if is_long else touch_area_lower
            if (is_long and price > buy_price) or (not is_long and price < buy_price):
                consecutive_touches[:] = -1
                count = 0
        
        prev_price = price
    return np.empty(0, dtype=np.int64)  # Return empty array instead of empty list

def calculate_touch_area(levels_by_date, is_long, df, symbol, market_hours, min_touches, bid_buffer_pct, use_median, touch_area_width_agg, multiplier, sell_time):
    current_id = 0
    touch_areas = []
    widths = []

    for date, levels in tqdm(levels_by_date.items()):
        market_open, market_close = market_hours.get(str(date), (None, None))
        if market_open and market_close and sell_time:
            market_close = min(
                datetime.strptime(str(date), '%Y-%m-%d') + timedelta(hours=sell_time.hour, minutes=sell_time.minute),
                market_close - timedelta(minutes=3)
            )
        
        if not market_open or not market_close:
            # print('Hours not available. Skipping',date)
            continue
        
        day_data = df[df.index.get_level_values('timestamp').date == date]
        day_timestamps = day_data.index.get_level_values('timestamp')
        day_timestamps_np = day_timestamps.astype(np.int64)  # Convert to nanoseconds
        day_prices = day_data['close'].values
        day_atr = day_data['MTR' if use_median else 'ATR'].values

        for (level_lower_bound, level_upper_bound, level), touches in levels.items():
            if len(touches) < min_touches:
                continue
            
            touch_timestamps_np = np.array([t.timestamp() * 1e9 for t in touches], dtype=np.int64)  # Convert to nanoseconds
            touch_indices = np.searchsorted(day_timestamps_np, touch_timestamps_np)
            
            touch_area_width = touch_area_width_agg(day_atr[touch_indices]) * multiplier

            if touch_area_width is None or np.isnan(touch_area_width) or touch_area_width <= 0:
                continue
            
            widths.append(touch_area_width)
            
            # SUBJECt TO CHANGE
            touch_area_lower = level - (2 * touch_area_width / 3) if is_long else level - (1 * touch_area_width / 3)
            touch_area_upper = level + (1 * touch_area_width / 3) if is_long else level + (2 * touch_area_width / 3)
            
            # touch_area_lower = level - (1 * touch_area_width / 3) if is_long else level - (2 * touch_area_width / 3)
            # touch_area_upper = level + (2 * touch_area_width / 3) if is_long else level + (1 * touch_area_width / 3)
            
            # touch_area_lower = level - (1 * touch_area_width / 2) if is_long else level - (1 * touch_area_width / 2)
            # touch_area_upper = level + (1 * touch_area_width / 2) if is_long else level + (1 * touch_area_width / 2)
            
            
            
            valid_mask = (day_timestamps[touch_indices] >= market_open) & (day_timestamps[touch_indices] < market_close)
            valid_touch_indices = touch_indices[valid_mask]
            valid_prices = day_prices[valid_touch_indices]
            
            consecutive_touch_indices = process_touches(
                valid_touch_indices, 
                valid_prices,
                touch_area_lower, 
                touch_area_upper,  
                level, 
                level_lower_bound,
                level_upper_bound, 
                is_long, 
                min_touches
            )
            
            if len(consecutive_touch_indices) == min_touches:
                consecutive_touches = day_timestamps[consecutive_touch_indices[consecutive_touch_indices != -1]]
                touch_area = TouchArea(
                    id=current_id,
                    level=level,
                    upper_bound=touch_area_upper,
                    lower_bound=touch_area_lower,
                    touches=consecutive_touches.tolist(),
                    is_long=is_long,
                    min_touches=min_touches,
                    bid_buffer_pct=bid_buffer_pct
                )
                touch_areas.append(touch_area)
                current_id += 1

    return touch_areas, widths

In [39]:
def calculate_touch_detection_area(symbol, start_date, end_date, atr_period=10, multiplier=2, min_touches=2, bid_buffer_pct=0.005, sell_time=None, use_median=False, touch_area_width_agg=np.median):
    """
    Calculate the touch detection area based on custom volatility analysis and identify strong support and resistance levels,
    considering only market hours and selling all positions before a specified time.

    Parameters:
    symbol (str|list): The stock symbol to analyze.
    start_date (str|datetime): The start datetime for historical data (format: 'YYYY-MM-DD HH:MM:SS.%f').
    end_date (str|datetime): The end datetime for historical data (format: 'YYYY-MM-DD HH:MM:SS.%f').
    atr_period (int): The period for calculating Average True Range (ATR).
    multiplier (float): The multiplier for ATR to define the touch detection area width.
    min_touches (int): The minimum number of touches to consider a level as strong support or resistance.
    bid_buffer_pct (float): The percentage above the high or below the low to place a stop market buy.
    sell_time (str): The time to sell all positions before (format: 'HH:MM'). If None or after market close, defaults to market close.
    use_median (bool): Flag to switch between using mean or median for levels and touch areas.
    touch_area_width_agg (function): Aggregation function for calculating touch_area_width based on ATR/MTR, which is based on use_median. Suggested: np.median (default), np.nanmax, np.mean
    
    Returns:
    dict: A dictionary with support and resistance areas for both long and short positions.
    """
    assert end_date > start_date
    
    
    # Alpaca API setup
    client = StockHistoricalDataClient(api_key=API_KEY, secret_key=API_SECRET)

    # Request historical data
    request_params = StockBarsRequest(
        symbol_or_symbols=symbol, # CAN be str or list. but set up for str (1 stock) for now)
        timeframe=TimeFrame.Minute,
        start=start_date if isinstance(start_date, datetime) else datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S.%f'),
        end=end_date if isinstance(end_date, datetime) else datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S.%f'),
        adjustment=Adjustment.ALL
    )
    bars = client.get_stock_bars(request_params)
    df = bars.df
    
    df.index = df.index.set_levels(pd.Series(df.index.get_level_values('timestamp').to_list()).map(lambda x: x.replace(tzinfo=None)), level='timestamp')
    df.sort_index(inplace=True)
    timestamps = df.index.get_level_values('timestamp')
    print(df.columns)
    # print(df)    
    
    df.reset_index().to_csv('bars.csv',index=False)

    # calculate_dynamic_levels(df, ema_short=9, ema_long=30) # default
    calculate_dynamic_levels(df)

    
    # Calculate True Range (TR)
    df['H-L'] = df['high'] - df['low']
    df['H-PC'] = np.abs(df['high'] - df['close'].shift(1))
    df['L-PC'] = np.abs(df['low'] - df['close'].shift(1))
    df['TR'] = df[['H-L', 'H-PC', 'L-PC']].max(axis=1)
    df['ATR'] = df['TR'].rolling(window=atr_period).mean()
    df['MTR'] = df['TR'].rolling(window=atr_period).median()
    
    
    # Group data by date
    grouped = df.groupby(timestamps.date)
    
    all_support_levels = defaultdict(dict)
    all_resistance_levels = defaultdict(dict)
    
    
    # def classify_level(level_items, index, day_df):
    #     return 'resistance' if touch_area_width_agg(level_items) > touch_area_width_agg(day_df.loc[index, 'central_value']) else 'support'
    
    def classify_level(level_items, index, day_df):
        return 'resistance' if level_items > day_df.loc[index, 'central_value'] else 'support'
    
    for date, day_df in tqdm(grouped):
        day_timestamps = day_df.index.get_level_values('timestamp')
        
        potential_levels = defaultdict(list)
        w_prev = 0
        
        for i in range(len(day_df)):
            high = day_df['high'].iloc[i]
            low = day_df['low'].iloc[i]
            close = day_df['close'].iloc[i]
            timestamp = day_timestamps[i]
            
            w = (high - low) / 2
            if w == 0:
                w = w_prev
            if w == 0:
                continue
            
            w_prev = w
            
            x = close - w
            y = close + w
            
            # Add this point to its own level
            potential_levels[(x, y)].append(timestamp)
            
            # Check if this point falls within any existing levels
            for (level_x, level_y), touches in potential_levels.items():
                if level_x <= close <= level_y and (level_x, level_y) != (x, y):
                    touches.append(timestamp)

        # Filter for strong levels
        strong_levels = {level: touches for level, touches in potential_levels.items() if len(touches) >= min_touches}

        # Classify levels as support or resistance
        for level, touches in strong_levels.items():
            initial_timestamp = touches[0]
            initial_close = day_df.loc[(symbol, initial_timestamp), 'close']
            
            # touch_prices = day_df.loc[(symbol, touches), 'close'].T.to_list()
            # classification = classify_level(touch_prices[0], (symbol, touches[0]), day_df)
            classification = classify_level(day_df.loc[(symbol, initial_timestamp), 'close'].item(), (symbol, initial_timestamp), day_df)
            
            # print(np.array([str(a) for a in touches]))
            
            if classification == 'support':
                all_support_levels[date][(level[0], level[1], initial_close)] = touches
            else:
                all_resistance_levels[date][(level[0], level[1], initial_close)] = touches

    
    unique_dates = pd.unique(timestamps.date)
    date_strs = [str(date) for date in unique_dates]
    market_hours = get_market_hours(date_strs)
    
    if sell_time:
        sell_time = datetime.strptime(sell_time, '%H:%M')



    # long_touch_area = calculate_touch_area(all_resistance_levels, True)
    # short_touch_area = calculate_touch_area(all_support_levels, False)
    # print(len(long_touch_area), len(short_touch_area))
    
    long_touch_area, long_widths = calculate_touch_area(
        all_resistance_levels, True, df, symbol, market_hours, min_touches, 
        bid_buffer_pct, use_median, touch_area_width_agg, multiplier, sell_time
    )
    short_touch_area, short_widths = calculate_touch_area(
        all_support_levels, False, df, symbol, market_hours, min_touches, 
        bid_buffer_pct, use_median, touch_area_width_agg, multiplier, sell_time
    )
    
    widths = long_widths + short_widths
    
    final_mask = pd.Series(False, index=df.index)
    for date in unique_dates:
        market_open, market_close = market_hours.get(str(date), (None, None))
        date = datetime.strptime(str(date), '%Y-%m-%d')
        if market_open is None or market_close is None:
            market_open = market_close = date
        elif sell_time:
            market_close = date + timedelta(hours=sell_time.hour, minutes=sell_time.minute)
        mask = (timestamps >= str(market_open)) & (timestamps <= str(market_close))
        final_mask |= mask

    return {
        'symbol': symbol,
        'long_touch_area': long_touch_area,
        'short_touch_area': short_touch_area,
        'market_hours': market_hours,
        'bars': df,
        'mask': final_mask,
        'bid_buffer_pct': bid_buffer_pct,
        'min_touches': min_touches,
        'sell_time': sell_time,
        'use_median': use_median
    }
    
    
# example usage:
# touch_detection_areas = calculate_touch_detection_area(symbol, start_date, end_date, atr_period=15, multiplier=1.6, min_touches=3, sell_time='16:00', use_median=True, touch_area_width_agg=np.median)
# balance, trades_executed, percent_increase, trades = backtest_strategy(touch_detection_areas)

In [40]:
def plot_touch_detection_areas(touch_detection_areas, zoom_start_date, zoom_end_date, save_path=None):
    symbol = touch_detection_areas['symbol']
    long_touch_area = touch_detection_areas['long_touch_area']
    short_touch_area = touch_detection_areas['short_touch_area']
    market_hours = touch_detection_areas['market_hours']
    df = touch_detection_areas['bars']
    mask = touch_detection_areas['mask']
    bid_buffer_pct = touch_detection_areas['bid_buffer_pct']
    min_touches = touch_detection_areas['min_touches']
    sell_time = touch_detection_areas['sell_time']
    use_median = touch_detection_areas['use_median']

        
    plt.figure(figsize=(14, 7))
    plt.plot(df.index.get_level_values('timestamp'), df['central_value'], label='central_value', color='yellow')
    plt.plot(df.index.get_level_values('timestamp'), df['close'], label='Close Price', color='blue')

    df = df[mask]
    timestamps = df.index.get_level_values('timestamp')

    # Prepare data structures for combined plotting
    scatter_data = defaultdict(lambda: defaultdict(list))
    fill_between_data = defaultdict(list)
    line_data = defaultdict(list)

    def find_area_end_idx(start_idx, area:TouchArea, market_open, market_close):
        assert market_open.date() == market_close.date()
        entry_price = area.get_buy_price
        
        for i in range(start_idx + 1, len(df)):
            current_time = timestamps[i]
            if current_time >= market_close:
                return i
            current_price = df['close'].iloc[i]
            
            if current_time >= area.get_min_touch_time:
                if area.is_long and current_price >= entry_price:
                    return i
                elif not area.is_long and current_price <= entry_price:
                    return i
            
        return len(df) - 1  # If no end condition is met, return the last index
    
    
    def process_area(area: TouchArea):
        if not area.is_active:
            return
        
        mark_pos = area.get_buy_price
        mark_shape = "v" if area.is_long else '^'
        color = 'red' if area.is_long else 'green'

        
        current_date = None
        for i, touch_time in enumerate(area.touches):
            if touch_time in timestamps:
                start_idx = timestamps.get_loc(str(touch_time))
                
                if timestamps[start_idx].date() != current_date:
                    current_date = timestamps[start_idx].date()
                    market_open, market_close = market_hours.get(str(current_date), (None, None))
                    if market_open and market_close and sell_time:
                        market_close = min(
                            datetime.strptime(str(current_date), '%Y-%m-%d') + timedelta(hours=sell_time.hour, minutes=sell_time.minute),
                            market_close - timedelta(minutes=3)
                        )
                
                end_idx = find_area_end_idx(start_idx, area, market_open, market_close)
                x1 = [timestamps[start_idx], area.get_min_touch_time]
                x2 = [area.get_min_touch_time, timestamps[end_idx]]
                
                if timestamps[end_idx] >= market_close:
                    continue
                scatter_color = 'gray' if i != min_touches - 1 else 'red' if end_idx == start_idx else 'blue'
                scatter_data[scatter_color][mark_shape].append((touch_time, mark_pos))
                
                if i == 0:  # first touch
                    fill_between_data[color].append((x1 + x2, [area.lower_bound] * 4, [area.upper_bound] * 4))
                    line_data['blue_alpha'].append((x1, [area.level] * 2))
                    line_data['blue'].append((x2, [area.level] * 2))

    for area in tqdm(long_touch_area + short_touch_area):
        process_area(area)

    # Plot combined data
    for color, shape_data in scatter_data.items():
        for shape, points in shape_data.items():
            if points:
                x, y = zip(*points)
                plt.scatter(x, y, color=color, s=12, marker=shape)

    for color, data in fill_between_data.items():
        for x, lower, upper in data:
            plt.fill_between(x[:2], lower[:2], upper[:2], color=color, alpha=0.1)
            plt.fill_between(x[2:], lower[2:], upper[2:], color=color, alpha=0.25)

    for color, data in line_data.items():
        for x, y in data:
            if color == 'blue_alpha':
                plt.plot(x, y, color='blue', linestyle='-', alpha=0.20)
            else:
                plt.plot(x, y, color='blue', linestyle='-')

    plt.title(f'{symbol} Price Chart with Touch Detection Areas')
    plt.xlabel('Date')
    plt.ylabel('Price')
    # plt.legend(['Close Price', 'Long Touch Area', 'Short Touch Area'])
    plt.legend().remove()
    plt.grid(True)
    
    zstart = datetime.strptime(zoom_start_date, '%Y-%m-%d %H:%M:%S.%f')
    zend = datetime.strptime(zoom_end_date, '%Y-%m-%d %H:%M:%S.%f')

    plt.xlim(max(zstart, timestamps[0]), min(zend, timestamps[-1]))

    ymin, ymax = 0, -1
    for i in range(len(timestamps)):
        if timestamps[i] >= zstart:
            print(timestamps[i])
            ymin = i-1
            break
    for i in range(len(timestamps)):
        if timestamps[i] >= zend:
            print(timestamps[i])
            ymax = i
            break
    ys = df['close'].iloc[max(ymin, 0):min(ymax, len(df))]
    plt.ylim(min(ys),max(ys))
    
    if save_path:
        plt.savefig(save_path)

    plt.show()

In [41]:
# Use the decimal module:
# For financial calculations where precision is crucial, consider using Python's decimal module, which provides exact decimal representations and arithmetic.
# https://docs.python.org/3/library/decimal.html

# from decimal import *
# getcontext()
# # Decimal(3.14)

In [65]:
def backtest_strategy(touch_detection_areas, initial_investment=10000, do_longs=True, do_shorts=True, use_margin=False, times_buying_power=4):
    symbol = touch_detection_areas['symbol']
    long_touch_area = touch_detection_areas['long_touch_area']
    short_touch_area = touch_detection_areas['short_touch_area']
    market_hours = touch_detection_areas['market_hours']
    df = touch_detection_areas['bars']
    mask = touch_detection_areas['mask']
    bid_buffer_pct = touch_detection_areas['bid_buffer_pct']
    min_touches = touch_detection_areas['min_touches']
    sell_time = touch_detection_areas['sell_time']
    use_median = touch_detection_areas['use_median']
    
    debug = True
    def debug_print(message):
        if debug:
            print(message)

    POSITION_OPENED = True
    NO_POSITION_OPENED = False
    
    all_touch_areas = []
    assert do_longs or do_shorts
    if do_longs:
        all_touch_areas.extend(long_touch_area)
    if do_shorts:
        all_touch_areas.extend(short_touch_area)
    touch_area_collection = TouchAreaCollection(all_touch_areas, min_touches)
        
    is_marginable = is_security_marginable(symbol)
    if not use_margin or not is_marginable:
        use_margin = False
        times_buying_power = 1
        
        
    print(f'{symbol} is {'NOT ' if not is_marginable else ''}marginable.')
    # return

    trades = []  # List to store all trades
    
    df = df[mask]
    df = df.sort_index(level='timestamp')
    timestamps = df.index.get_level_values('timestamp')

    balance = initial_investment
    total_account_value = initial_investment
    
    trades_executed = 0
    open_positions = {}
    current_id = 0
    

    def update_total_account_value(current_price):
        nonlocal total_account_value, balance
        for position in open_positions.values():
            position.update_market_value(current_price)
        
        cash_values = sum(position.cash_value() for position in open_positions.values())
        equity_values = sum(position.equity_value() for position in open_positions.values())
        
        total_account_value = balance + cash_values + equity_values
        
        print(f"  update_total_account_value:")
        print(f"    balance: {balance:.6f}")
        print(f"    cash_values: {cash_values:.6f}")
        print(f"    equity_values: {equity_values:.6f}")
        print(f"    total_account_value: {total_account_value:.6f}")
        for area_id, position in open_positions.items():
            print(f"      Position {area_id}: Shares: {position.current_shares}, Market Value: {position.current_market_value:.6f}, Cash Used: {position.cash_value():.6f}")

                        
    def rebalance(add: float = None, set: float = None, current_price: float = None):
        nonlocal balance, total_account_value
        old_balance = balance
        if add is not None and set is None:
            balance += add
        elif set is not None and add is None:
            balance = set

        assert balance >= 0, f"Balance became negative: {balance}"

        if current_price is not None:
            update_total_account_value(current_price)
        print(f"Rebalance: Old balance: {old_balance:.4f}, Change: {(balance - old_balance):.4f}, New balance: {balance:.4f}, Total Account Value: {total_account_value:.4f}")
                        
    def exit_action(area_id, position):
        nonlocal trades, balance, total_account_value
        
        exit_price = position.current_stop_price
        if position.is_long:
            profit_loss = (exit_price - position.entry_price) * position.current_shares
        else:
            profit_loss = (position.entry_price - exit_price) * position.current_shares
        
        cash_to_add = position.initial_cash_used + (profit_loss / position.actual_margin_multiplier)
        
        print(f"  exit_action:")
        print(f"    entry_price: {position.entry_price:.4f}")
        print(f"    exit_price: {exit_price:.4f}")
        print(f"    shares: {position.current_shares}")
        print(f"    initial_cash_used: {position.initial_cash_used:.4f}")
        print(f"    profit_loss: {profit_loss:.4f}")
        print(f"    cash_to_add: {cash_to_add:.4f}")

        rebalance(add=cash_to_add, current_price=exit_price)

        print(f"{'res' if position.is_long else 'sup'} area {area_id}:\t{position.id} {position.sub_positions[-1].exit_time} - Exit {'Long ' if position.is_long else 'Short'} at {exit_price:.4f} "
            f"({'+'if position.price_diff >= 0 else ''}{position.price_diff:.4f}). "
            f"Net {'+'if profit_loss >= 0 else ''}{profit_loss / position.actual_margin_multiplier:.4f} "
            f"(ROE: {(profit_loss / position.initial_cash_used) * 100:.4f}%, After -{position.total_transaction_costs:.4f} fees)")
        trades.append(position)
        
        
    def close_all_positions(timestamp, exit_price):
        nonlocal trades_executed
        if not open_positions:
            return
        
        positions_to_remove = []

        for area_id, position in list(open_positions.items()):
            position.close(timestamp, exit_price)
            trades_executed += 1
            position.area.record_entry_exit(position.entry_time, position.entry_price, 
                                            timestamp, exit_price)
            position.area.terminate(touch_area_collection)
            positions_to_remove.append(area_id)

        temp = {}
        for area_id in positions_to_remove:
            temp[area_id] = open_positions[area_id]
            del open_positions[area_id]
        for area_id in positions_to_remove:
            exit_action(area_id, temp[area_id])

        assert not open_positions
        open_positions.clear()

    def calculate_max_position_size():
        nonlocal total_account_value
        
        if use_margin and is_marginable:
            initial_margin_requirement = 0.5  # 50% for marginable securities
            overall_margin_multiplier = min(times_buying_power, 4)
            actual_margin_multiplier = min(overall_margin_multiplier, 2.0)
        else:
            initial_margin_requirement = 1.0  # 100% for non-marginable securities
            overall_margin_multiplier = 1.0
            actual_margin_multiplier = 1.0

        available_margin = balance * overall_margin_multiplier
        used_margin = sum(p.initial_balance for p in open_positions.values())
        available_buying_power = available_margin - used_margin
        
        max_position_size = available_buying_power / initial_margin_requirement

        return max_position_size, actual_margin_multiplier, overall_margin_multiplier, initial_margin_requirement

    def place_stop_market_buy(area: TouchArea, timestamp: datetime, open_price: float, high_price: float, low_price: float, close_price: float, prev_close: float):
        nonlocal balance, current_id, total_account_value, open_positions, trades_executed

        if open_positions or balance <= 0:
            return NO_POSITION_OPENED

        debug_print(f"Attempting order: {'Long' if area.is_long else 'Short'} at {area.get_buy_price:.4f}")
        debug_print(f"  Balance: {balance:.4f}, Total Account Value: {total_account_value:.4f}")


        # Check if the stop buy would have executed based on high/low. close price unnecessary (i think)
        if area.is_long:
            if prev_close > area.get_buy_price:
                debug_print(f"  Rejected: Previous close ({prev_close:.4f}) above buy price, likey re-entering area ({area.get_buy_price:.4f})")
                return NO_POSITION_OPENED
            if high_price < area.get_buy_price:
                debug_print(f"  Rejected: High price ({high_price:.4f}) didn't reach buy price ({area.get_buy_price:.4f})")
                return NO_POSITION_OPENED
        else:  # short
            if prev_close < area.get_buy_price:
                debug_print(f"  Rejected: Previous close ({prev_close:.4f}) below buy price, likey re-entering area ({area.get_buy_price:.4f})")
                return NO_POSITION_OPENED
            if low_price > area.get_buy_price:
                debug_print(f"  Rejected: Low price ({low_price:.4f}) didn't reach buy price ({area.get_buy_price:.4f})")
                return NO_POSITION_OPENED

        # Stop buy (placed at time of min_touches) would have executed ############### IMPORTANT
        execution_price = area.get_buy_price
        
        # # If not using stop buys and only going by bars
        # if area.is_long:
        #     execution_price = max(close_price, area.get_buy_price)
        # else:
        #     execution_price = min(close_price, area.get_buy_price)

        # Calculate position size, etc...
        max_position_size, actual_margin_multiplier, overall_margin_multiplier, initial_margin_requirement = calculate_max_position_size()

        # Calculate total shares and round down to whole number
        total_shares = math.floor((balance * overall_margin_multiplier) / execution_price)
        # if total_shares == 0:
        #     return NO_POSITION_OPENED
        
        # # If ensuring even number of stocks:
        # if total_shares % 2 == 1:
        #     total_shares -= 1
            
        # Recalculate invest_amount based on rounded total_shares
        invest_amount = total_shares * execution_price
        actual_cash_used = invest_amount / overall_margin_multiplier
        
        if actual_cash_used > balance:
            debug_print(f"  Order rejected: Insufficient balance ({actual_cash_used:.4f} > {balance:.4f})")
            return NO_POSITION_OPENED

        debug_print(f"  Shares: {total_shares}, Invest Amount: {invest_amount:.4f}")
        debug_print(f"  Margin Multiplier: {actual_margin_multiplier:.2f}")
        
        debug_print(f"  execution_price: {execution_price:.2f}")
        
        
        # Create the position
        position = TradePosition(
            id=current_id,
            area=area,
            is_long=area.is_long,
            entry_time=timestamp,
            initial_balance=actual_cash_used,
            initial_cash_used=actual_cash_used,
            current_market_value=invest_amount,
            initial_shares=total_shares,
            current_shares=0,
            entry_price=execution_price,
            use_margin=use_margin,
            is_marginable=is_marginable,
            times_buying_power=times_buying_power,
            actual_margin_multiplier=actual_margin_multiplier,
            current_stop_price=high_price - area.get_range if area.is_long else low_price + area.get_range,
            max_price=high_price if area.is_long else None,
            min_price=low_price if not area.is_long else None,
            # last_price=execution_price
        )
        
        # Create sub-positions
        shares_per_sub = position.shares_per_sub
        for shares in shares_per_sub:
            position.add_sub_position(timestamp, execution_price, shares)
            # position.add_shares(shares)  # Update current_shares as we add sub-positions
        
        debug_print(f"  position.entry_price: {position.entry_price:.4f}")
        
        current_id += 1
        
        # position.update_stop_price(close_price)

        # # If we need to immediately exit the position if close price is already less than the buy
        # # Initial tests - poor results. probably dont use
        
        # if (area.is_long and close_price <= area.get_buy_price) or (not area.is_long and close_price >= area.get_buy_price):
        #     # Immediate exit
        #     rebalance(add=-actual_cash_used) # For emulation sake
            
        #     position.update_stop_price(close_price)
        #     position.close(timestamp, close_price)
        #     trades_executed += 1
        #     position.area.record_entry_exit(timestamp, execution_price, timestamp, close_price)
            
        #     cash_to_add = position.initial_balance + position.profit_loss
        #     rebalance(add=cash_to_add, current_price=close_price)
            
        #     temp = f"{high_price:.4f} high" if area.is_long else f"{low_price:.4f} low"
            
        #     print(f"{'res' if area.is_long else 'sup'} area {area.id}: {current_id} {timestamp} - Enter and Immediate Exit {'Long ' if area.is_long else 'Short'} at {close_price:.4f} close after {temp} "
        #         f"({'+'if position.price_diff >= 0 else ''}{position.price_diff:.4f}). "
        #         f"Net {'+'if position.profit_loss >= 0 else ''}{position.profit_loss:.4f} "
        #         f"(ROE: {position.return_on_equity:.2f}%, After -{position.total_transaction_costs:.4f} fees)")

        #     trades.append(position)
        #     return POSITION_OPENED # need to do POSITION_OPENED (True), otherwise the other touch areas will encounter the same and thus cause a string of losses
        # else:
        
        # Add to open positions
        open_positions[area.id] = position
        
        print(f"{'res' if area.is_long else 'sup'} area {area.id}: {current_id} {timestamp} - Enter {'Long ' if area.is_long else 'Short'} at {execution_price:.4f}. "
              f"Shares: {total_shares}, Amount: ${invest_amount:.4f} (Margin: {actual_margin_multiplier:.2f}x, Overall: {overall_margin_multiplier:.2f}x, Sub-positions: {len(position.sub_positions)})")

        rebalance(add=-actual_cash_used, current_price=close_price)
        return POSITION_OPENED


    def update_positions(timestamp, open_price, high_price, low_price, close_price):
        nonlocal trades_executed
        positions_to_remove = []

        def perform_exit(area_id, position):
            nonlocal trades_executed
            position.close(timestamp, position.current_stop_price)
            trades_executed += 1
            position.area.record_entry_exit(position.entry_time, position.entry_price, 
                                            timestamp, position.current_stop_price)
            position.area.terminate(touch_area_collection)
            positions_to_remove.append(area_id)
            

        def calculate_target_shares_percentage(position, current_price):
            price_movement = abs(current_price - position.current_stop_price)
            target_percentage = min(price_movement / position.area.get_range, 1)
            # debug_print(f'price_movement = abs({current_price}-{position.current_stop_price})')
            # debug_print(f'target_percentage = {price_movement}/{position.area.get_range}')
            
            return target_percentage

        for area_id, position in open_positions.items():
            # old_stop_price = position.current_stop_price
            position.update_last_price(close_price)

            debug_print(f"Updating position {position.id} at {timestamp}")
            debug_print(f"Current shares: {position.current_shares}, Initial shares: {position.initial_shares}")
            debug_print(f"Current stop price: {position.current_stop_price:.4f}")
            
        
            # OHLC logic for trailing stops
            position.update_stop_price(open_price)
            debug_print(f"  Stop price at open: {position.current_stop_price:.4f} ")
            if position.should_exit(open_price): 
                debug_print(f"  Trailing Stop - Exited at open: {open_price:.4f}")
                perform_exit(area_id, position)
                continue
            
            # If not stopped out at open, simulate intra-minute price movement
            if position.is_long:
                # For long positions, the stop moves up if high price increases
                position.update_stop_price(high_price)
                debug_print(f"  Stop price at high: {position.current_stop_price:.4f} ")
                # Check if low price hit the stop
                if position.should_exit(high_price):
                    debug_print(f"  Trailing Stop - Exited at high: {high_price:.4f}")
                    perform_exit(area_id, position)
                    continue
            else:
                # For short positions, the stop moves down if low price decreases
                position.update_stop_price(low_price)
                debug_print(f"  Stop price at low: {position.current_stop_price:.4f} ")
                # Check if high price hit the stop
                if position.should_exit(low_price):
                    debug_print(f"  Trailing Stop - Exited at low: {low_price:.4f}")
                    perform_exit(area_id, position)
                    continue

            position.update_stop_price(close_price)
            debug_print(f"  Stop price at close: {position.current_stop_price:.4f} ")
            if position.should_exit(close_price):
                debug_print(f"  Trailing Stop - Exited at close: {close_price:.4f}")
                perform_exit(area_id, position)
                continue


            # Partial exit and entry logic
            target_percentage = calculate_target_shares_percentage(position, close_price)
            current_percentage = position.current_shares / position.initial_shares
            
            if target_percentage != current_percentage:
                debug_print(f'current_percentage = {position.current_shares}/{position.initial_shares}')
                debug_print(f"  Current -> Target percentage: {current_percentage*100}% -> {target_percentage*100}%")
            
            if target_percentage < current_percentage:
                # Need to reduce position size
                shares_to_adjust = math.floor(position.current_shares * (current_percentage - target_percentage))
                if shares_to_adjust > 0:
                    realized_pnl, cash_generated = position.partial_exit(timestamp, close_price, shares_to_adjust)
                    rebalance(add=cash_generated, current_price=close_price)
                    print(f"Partial exit: Sold {shares_to_adjust} shares at {close_price:.4f}, Realized PnL: {realized_pnl:.4f}, Cash added: {cash_generated:.4f}")

                    
            elif target_percentage > current_percentage:
                # Need to increase position size
                shares_to_adjust = math.floor(position.initial_shares * (target_percentage - current_percentage))
                if shares_to_adjust > 0:
                    cash_needed = shares_to_adjust * close_price
                    if balance >= cash_needed:
                        position.add_sub_position(timestamp, close_price, shares_to_adjust)
                        rebalance(add=-cash_needed, current_price=close_price)
                        print(f"Partial enter: Bought {shares_to_adjust} shares at {close_price:.4f}")
                    
        temp = {}
        for area_id in positions_to_remove:
            temp[area_id] = open_positions[area_id]
            del open_positions[area_id]
        for area_id in positions_to_remove:
            exit_action(area_id, temp[area_id])
        
        # update_total_account_value(close_price)
        if positions_to_remove:
            debug_print(f"  Updated Total Account Value: {total_account_value:.4f}")

    # print(f"START\nStrategy: {'Long' if do_longs else ''}{'&' if do_longs and do_shorts else ''}{'Short' if do_shorts else ''}.")
    debug_print(f"Strategy: {'Long' if do_longs else ''}{'&' if do_longs and do_shorts else ''}{'Short' if do_shorts else ''}")
    print(f"{timestamps[0]} -> {timestamps[-1]}")
    # print('Initial Investment:', initial_investment)
    debug_print(f"Initial Investment: {initial_investment}, Times Buying Power: {times_buying_power}")
    print('Number of touch areas:', len(all_touch_areas))
    

    daily_data = None
    current_date, market_open, market_close = None, None, None
    for i in tqdm(range(1, len(timestamps))):
        current_time = timestamps[i]
        print(current_time)
        # print(current_time, len(open_positions))
        
        if current_time.date() != current_date:
            debug_print(f"\nNew trading day: {current_time.date()}")
            # New day, reset daily data
            current_date = current_time.date()
            market_open, market_close = market_hours.get(str(current_date), (None, None))
            if market_open and market_close:
                if sell_time:
                    market_close = min(
                        datetime.strptime(str(current_date), '%Y-%m-%d') + timedelta(hours=sell_time.hour, minutes=sell_time.minute),
                        market_close - timedelta(minutes=3)
                    )
            daily_data = df[timestamps.date == current_date]
            daily_index = 1 # start at 2nd position
            assert not open_positions
        
        if not market_open or not market_close:
            continue

        if market_open <= current_time < market_close and i != len(df)-1:
            # debug_print(f"\n{current_time} - Market Open")
            
            prev_close = daily_data['close'].iloc[daily_index - 1]
            open_price = daily_data['open'].iloc[daily_index]
            high_price = daily_data['high'].iloc[daily_index]
            low_price = daily_data['low'].iloc[daily_index]
            close_price = daily_data['close'].iloc[daily_index]

            
            update_positions(current_time, open_price, close_price, high_price, low_price)
            
            active_areas = touch_area_collection.get_active_areas(current_time)
            for area in active_areas:
                if balance <= 0:
                    break
                if open_positions:
                    break
                
                if (area.is_long and do_longs) or (not area.is_long and do_shorts):

                    if place_stop_market_buy(area, current_time, open_price, high_price, low_price, close_price, prev_close):
                        break  # Exit the loop after placing a position
            
            daily_index += 1
        elif current_time >= market_close or i >= len(df)-1:
            debug_print(f"\n{current_time} - Market Close")
            close_all_positions(current_time, df['close'].iloc[i])

        # print(len(open_positions))
    if current_time >= market_close:
        assert not open_positions

    percent_increase = ((balance - initial_investment) / initial_investment) * 100

    # Buy and hold strategy
    start_price = df['close'].iloc[0]
    end_price = df['close'].iloc[-1]
    buy_and_hold_increase = ((end_price - start_price) / start_price) * 100

    # export_trades_to_csv(trades,'trades_output.csv')
    
    total_borrow_fees = sum(trade.stock_borrow_cost for trade in trades)
    
    total_profit_loss = sum(trade.profit_loss for trade in trades)
    total_transaction_costs = sum(trade.total_transaction_costs for trade in trades)
    
    total_entry_transaction_costs = sum(trade.entry_transaction_costs for trade in trades)
    total_exit_transaction_costs = sum(trade.exit_transaction_costs for trade in trades)
    total_stock_borrow_costs = sum(trade.stock_borrow_cost for trade in trades)
    
    average_profit_loss = total_profit_loss / len(trades) if trades else 0
    average_profit_loss_percentage = sum(trade.profit_loss_percentage for trade in trades) / len(trades) if trades else 0
    winning_trades = sum(1 for trade in trades if trade.profit_loss > 0)
    losing_trades = sum(1 for trade in trades if trade.profit_loss < 0)
    avg_sub_pos = np.mean([trade.num_sub_positions for trade in trades])

    assert trades_executed == len(trades)
    
    debug_print("\nBacktest Complete")
    debug_print(f"Final Balance: {balance:.2f}")
    debug_print(f"Total Trades Executed: {trades_executed}")
    debug_print(f"Win Rate: {winning_trades / len(trades) * 100:.2f}%")

    # Print statistics
    print(f"END\nStrategy: {'Long' if do_longs else ''}{'&' if do_longs and do_shorts else ''}{'Short' if do_shorts else ''}")
    print(f"{timestamps[0]} -> {timestamps[-1]}")

    print("\nOverall Statistics:")
    print('Initial Investment:', initial_investment)
    print('Final Balance:', balance)
    print(f"Percent increase: {percent_increase:.4f}% ***")
    print(f"If held baseline: {buy_and_hold_increase:.4f}%")
    print('Number of Trades Executed:', trades_executed)
    print(f"\nTotal Profit/Loss (including fees): ${total_profit_loss:.2f}")
    print(f"Total Transaction Costs: ${total_transaction_costs:.2f}")
    print(f"\nBorrow Fees: ${total_borrow_fees:.2f}")
    print(f"Average Profit/Loss per Trade (including fees): ${average_profit_loss:.2f}")
    print(f"Average Profit/Loss Percentage per Trade (including fees): {average_profit_loss_percentage:.2f}%")
    print(f"Average Return on Equity per Trade: {sum(trade.return_on_equity for trade in trades) / len(trades):.2f}%")
    print(f"Number of Winning Trades: {winning_trades}")
    print(f"Number of Losing Trades: {losing_trades}")
    print(f"Win Rate: {winning_trades / len(trades) * 100:.2f}%" if trades else "Win Rate: N/A")
    print(f"\nMargin Usage:")
    print(f"Margin Enabled: {'Yes' if use_margin else 'No'}")
    print(f"Max Buying Power: {times_buying_power}x")
    print(f"Average Margin Multiplier: {sum(trade.margin_multiplier for trade in trades) / len(trades):.2f}x")
    print(f"Average Sub Positions: {avg_sub_pos:.2f}")
    
    
    # return balance, trades_executed, percent_increase, trades
    # return balance, trades_executed, percent_increase, winning_trades, losing_trades, winning_trades / len(trades) * 100
    return balance, trades_executed, percent_increase, winning_trades / len(trades) * 100, total_entry_transaction_costs, total_exit_transaction_costs, \
        total_stock_borrow_costs, total_transaction_costs, avg_sub_pos

In [66]:
# symbol = "NVDA"
# symbol = "MSFT"
# symbol = "AAPL"
# symbol = "TJX"
# symbol = "JNJ"
symbol = "AMZN"
# symbol = "AZN"
# symbol = "TSLA"

# start_date = "2024-06-03 00:00:00.0"
# end_date =   "2024-06-04 00:00:00.0"

# end_date = "2024-03-30 00:00:00.0"


# start_date = "2024-02-01 00:00:00.0"
# end_date   = "2024-03-06 00:00:00.0"
start_date = "2024-02-01 00:00:00.0"
end_date   = "2024-02-01 12:00:00.0"
# start_date = "2024-02-01 00:00:00.0"
# end_date   = "2024-02-02 00:00:00.0"


# start_date = "2023-07-07 00:00:00.0"
# end_date =   "2024-07-07 00:00:00.0"


# start_date = "2023-07-10 00:00:00.0"
# end_date =   "2024-06-26 00:00:00.0"

touch_detection_areas = calculate_touch_detection_area(symbol, start_date, end_date, atr_period=15, multiplier=1.6, min_touches=3, sell_time='16:00', use_median=True, touch_area_width_agg=np.median)


Index(['open', 'high', 'low', 'close', 'volume', 'trade_count', 'vwap'], dtype='object')


100%|██████████| 1/1 [00:00<00:00, 205.89it/s]


In [67]:

# zoom_start_date = "2024-04-15 00:00:00.0"
# zoom_end_date = "2024-04-15 21:00:00.0"

# i, j = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S.%f'), datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S.%f')

# while i <= j:
#     print(i)
#     zoom_start_date = str(i + timedelta(hours=8))
#     zoom_end_date = str(i + timedelta(hours=20))

#     plot_touch_detection_areas(touch_detection_areas, symbol, zoom_start_date, zoom_end_date)
#     i += timedelta(days=1)

zoom_start_date = start_date
zoom_end_date = end_date
# zoom_start_date = "2024-06-03 14:45:00.0"
# zoom_end_date =   "2024-06-04 16:00:00.0"
# plot_touch_detection_areas(touch_detection_areas, zoom_start_date, zoom_end_date)

In [68]:

# # Usage examples:
# print("Both Long and Short:")
backtest_strategy( touch_detection_areas, use_margin=True, do_longs=True, do_shorts=True, times_buying_power=1)

# # print("\nLong only:")
# backtest_strategy( touch_detection_areas, use_margin=True, do_longs=True, do_shorts=False, times_buying_power=2)
  
# # print("\nShort only:")
# backtest_strategy( touch_detection_areas, use_margin=True, do_longs=False, do_shorts=True, times_buying_power=2)


AMZN is marginable.
Strategy: Long&Short
2024-02-01 09:32:00 -> 2024-02-01 12:00:00
Initial Investment: 10000, Times Buying Power: 1
Number of touch areas: 18


100%|██████████| 55/55 [00:00<00:00, 7857.98it/s]

2024-02-01 09:33:00

New trading day: 2024-02-01
2024-02-01 09:34:00
2024-02-01 09:35:00
2024-02-01 09:36:00
2024-02-01 09:39:00
2024-02-01 09:40:00
2024-02-01 09:42:00
Attempting order: Short at 156.3733
  Balance: 10000.0000, Total Account Value: 10000.0000
  Rejected: Low price (156.3900) didn't reach buy price (156.3733)
2024-02-01 09:43:00
Attempting order: Short at 156.3733
  Balance: 10000.0000, Total Account Value: 10000.0000
  Shares: 63, Invest Amount: 9851.5200
  Margin Multiplier: 1.00
  execution_price: 156.37
buying 63 shares at 2024-02-01 09:43:00, 156.3733 (9851.5200)
add_transaction 2024-02-01 09:43:00, 63, 156.3733, True, 0.2356
  position.entry_price: 156.3733
sup area 6: 1 2024-02-01 09:43:00 - Enter Short at 156.3733. Shares: 63, Amount: $9851.5200 (Margin: 1.00x, Overall: 1.00x, Sub-positions: 1)
  update_total_account_value:
    balance: 148.480000
    cash_values: 9851.520000
    equity_values: -1.470000
    total_account_value: 9998.530000
      Position 6: Sha


Backtest Complete
Final Balance: 51531.42
Total Trades Executed: 5
Win Rate: 40.00%
END
Strategy: Long&Short
2024-02-01 09:32:00 -> 2024-02-01 12:00:00

Overall Statistics:
Initial Investment: 10000
Final Balance: 51531.41999999999
Percent increase: 415.3142% ***
If held baseline: 0.0447%
Number of Trades Executed: 5

Total Profit/Loss (including fees): $-13.47
Total Transaction Costs: $5.11

Borrow Fees: $0.00
Average Profit/Loss per Trade (including fees): $-2.69
Average Profit/Loss Percentage per Trade (including fees): -0.01%
Average Return on Equity per Trade: -0.01%
Number of Winning Trades: 2
Number of Losing Trades: 3
Win Rate: 40.00%

Margin Usage:
Margin Enabled: Yes
Max Buying Power: 1x
Average Margin Multiplier: 1.00x
Average Sub Positions: 1.00


(np.float64(51531.41999999999),
 5,
 np.float64(415.3141999999999),
 40.0,
 np.float64(2.922808477266666),
 np.float64(2.1839651879999997),
 np.float64(0.004185104861111111),
 np.float64(5.106773665266665),
 np.float64(1.0))

In [69]:
# balance, trades_executed, percent_increase, winning_trades, losing_trades, winning_trades / len(trades) * 100
# return balance, trades_executed, percent_increase, winning_trades / len(trades) * 100, total_entry_transaction_costs, total_exit_transaction_costs, total_stock_borrow_costs, total_transaction_costs

from IPython.utils import io
results = pd.DataFrame()
for i in list(np.arange(0.1, 4.01, 0.1)):
    with io.capture_output() as captured:
        balance, trades_executed, percent_increase, winrate, entry_costs, exit_costs, borrow_costs, total_costs, avg_sub_pos = backtest_strategy( touch_detection_areas, use_margin=True, do_longs=True, do_shorts=True, times_buying_power=i)
        newrow = {'times_buying_power':f'{i:.1f}', 'balance':[f'{balance:.2f}'], 'trades_executed':[trades_executed], 'percent_increase_per_trade':[f'{percent_increase/trades_executed:.6f}'], \
            'winrate':f'{winrate:.6f}', 'entry_costs_per_trade':f'{entry_costs/trades_executed:.6f}', 'exit_costs_per_trade':f'{exit_costs/trades_executed:.6f}', \
                'borrow_costs_per_trade':f'{borrow_costs/trades_executed:.6f}', 'total_costs_per_trade':f'{total_costs/trades_executed:.6f}', 'avg_sub_pos':f'{avg_sub_pos:.2f}'}
        results = pd.concat([results, pd.DataFrame.from_dict(newrow)],ignore_index=True)
results

,times_buying_power,balance,trades_executed,percent_increase_per_trade,winrate,entry_costs_per_trade,exit_costs_per_trade,borrow_costs_per_trade,total_costs_per_trade,avg_sub_pos
0,0.1,85662.83,5,151.325653,40.000000,0.070918,0.056588,0.000085,0.127505,1.00
1,0.2,90834.96,5,161.669927,40.000000,0.132594,0.105219,0.000179,0.237813,1.00
2,0.3,90262.69,5,160.525373,40.000000,0.197855,0.154003,0.000279,0.351858,1.00
3,0.4,84808.20,5,149.616403,40.000000,0.255946,0.197765,0.000362,0.453710,1.00
4,0.5,79405.81,5,138.811621,40.000000,0.313793,0.240809,0.000449,0.554602,1.00
5,0.6,74238.34,5,128.476680,40.000000,0.373241,0.284760,0.000527,0.658002,1.00
6,0.7,67839.96,5,115.679910,40.000000,0.425638,0.322351,0.000611,0.747988,1.00
7,0.8,62415.21,5,104.830413,40.000000,0.482477,0.362835,0.000688,0.845312,1.00
8,0.9,56559.87,5,93.119741,40.000000,0.532336,0.398248,0.000761,0.930585,1.00
9,1.0,51531.42,5,83.062840,40.000000,0.584562,0.436793,0.000837,1.021355,1.00


In [70]:
results.to_csv('times_buying_power_test.csv',index=False)

In [71]:
results.astype(float).describe()

,times_buying_power,balance,trades_executed,percent_increase_per_trade,winrate,entry_costs_per_trade,exit_costs_per_trade,borrow_costs_per_trade,total_costs_per_trade,avg_sub_pos
count,40.000000,40.000000,40.0,40.000000,40.0,40.000000,40.000000,40.000000,40.000000,40.000000
mean,2.050000,79670.339500,5.0,139.340678,40.0,1.293920,0.888478,0.008750,2.182398,1.760000
std,1.169045,33640.517495,0.0,67.281035,0.0,0.979814,0.671191,0.008252,1.650868,0.793822
min,0.100000,33044.370000,5.0,46.088740,40.0,0.070918,0.056588,0.000085,0.127505,1.000000
25%,1.075000,53422.917500,5.0,86.845834,40.0,0.563916,0.389763,0.002025,0.950060,1.000000
50%,2.050000,74041.055000,5.0,128.082108,40.0,0.922162,0.622790,0.006031,1.544952,1.500000
75%,3.025000,94975.422500,5.0,169.950842,40.0,1.934714,1.320891,0.014046,3.255605,2.600000
max,4.000000,159546.680000,5.0,299.093354,40.0,3.542796,2.446016,0.027764,5.988812,3.000000
